In [1]:
%%capture
!pip install pygame
!pip install pygad

!git clone https://github.com/karinemiras/evoman_framework.git tmp
!cp -r /kaggle/working/tmp/* /kaggle/working/
!rm -R /kaggle/working/tmp

In [2]:
import sys, os
import numpy as np
import pandas as pd

import random

from evoman.environment import Environment
from evoman.controller import Controller
from demo_controller import player_controller
from deap import base, creator, tools, algorithms
import multiprocessing
import optuna

import seaborn as sns
import matplotlib.pyplot as plt

pygame 2.5.2 (SDL 2.28.2, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [5]:
def parameter_count(hidden_neurons):
    if hidden_neurons>0:
        n_w = (20*hidden_neurons) + (hidden_neurons*5)
        n_b = hidden_neurons + 5
        return n_w + n_b
    else:
        return (20*5)+5
    
def fitness_function(solution):
    sum_gain = 0
    for enemy in range(1,9):
        env.update_parameter("enemies",[enemy])
        res = env.play(np.array(solution))
        gain = res[1]-res[2]
        sum_gain += gain
    return (sum_gain,)

"""
def fitness_function(solution):
    defeated = 0
    for enemy in range(1,9):
        env.update_parameter("enemies",[enemy])
        res = env.play(np.array(solution))
        if res[2]<=0:
            defeated += 1
    return (defeated,)
"""

'\ndef fitness_function(solution):\n    defeated = 0\n    for enemy in range(1,9):\n        env.update_parameter("enemies",[enemy])\n        res = env.play(np.array(solution))\n        if res[2]<=0:\n            defeated += 1\n    return (defeated,)\n'

In [6]:
def cxWholeAritmetic(parent1,parent2,alpha):
    child1 = (alpha*np.array(parent1))+((1-alpha)*np.array(parent2))
    child2 = (alpha*np.array(parent2))+((1-alpha)*np.array(parent1))
    return creator.Individual(child1),creator.Individual(child2)

def cxBlend(parent1,parent2,alpha):
    gamma = (1+2*alpha)*np.random.uniform(size=(len(parent1,))) - alpha
    child1 = (gamma*np.array(parent1))+((1-gamma)*np.array(parent2))
    child2 = (gamma*np.array(parent2))+((1-gamma)*np.array(parent1))
    return creator.Individual(child1),creator.Individual(child2)

In [7]:
def mutGaussian(ind,sigma,indpb):
    mutation = [np.random.normal(0,sigma) if np.random.uniform()<indpb else 0 for i in range(len(ind))]
    return creator.Individual(np.array(ind) + np.array(mutation))

def mutGaussianAdaptive(ind,sigma,tau,indpb):
    new_sigma = sigma * np.exp(np.random.normal(0,tau,size=(len(ind),)))
    new_sigma[new_sigma<0.1] = 0.1
    mutation = [np.random.normal(0,s) if np.random.uniform()<indpb else 0 for s in new_sigma]
    return creator.Individual(np.array(ind) + np.array(mutation))

In [8]:
def genetic_algorithm(initial_range,pop_size,n_offspring,tournsize_parent,cxpb,cx_type,
                      alpha,mutpb,mut_type,sigma,indpb,tau,tournsize_survival,ngen,verbose):
    
    global individual_size,env,creator,stats
    
    toolbox = base.Toolbox()
    toolbox.register("weight_bin", np.random.uniform,-initial_range,initial_range)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.weight_bin, n=individual_size)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("evaluate", fitness_function)

    pop = toolbox.population(n=pop_size)
    hof = {'best_solution':-np.inf,'max_value':-np.inf}

    with multiprocessing.Pool() as pool:
        # evaluate initial population
        for ind, fit in zip(pop, pool.map(toolbox.evaluate,pop)):
            ind.fitness.values = fit

        # record best solution
        for ind in pop:
            if ind.fitness.values[0]>hof['max_value']:
                hof['best_solution'] = ind
                hof['max_value'] = ind.fitness.values[0]

        logbook = tools.Logbook()
        logbook.header = ['gen', 'nevals'] + (stats.fields if stats else []) + ['max_hof']
        record = stats.compile(pop) if stats else {}

        logbook.record(gen=0, nevals=len(pop), **record, max_hof=hof['max_value'])
        if verbose==1:
            print(logbook.stream)

        for g in range(ngen):
            # select parents
            parents = tools.selTournament(individuals=pop,k=n_offspring*pop_size,tournsize=tournsize_parent)

            # create offspring
            offspring = []
            for child1, child2 in zip(parents[::2], parents[1::2]):
                if np.random.uniform() < cxpb:
                    if cx_type=="whole_aritmetic":
                        if alpha=="random":
                            alpha=np.random.uniform()
                            child1,child2 = cxWholeAritmetic(child1, child2, alpha)
                        else:
                            child1,child2 = cxWholeAritmetic(child1, child2, alpha)
                    elif cx_type=="blend":
                        if alpha=="random":
                            alpha=np.random.uniform()
                            child1,child2 = cxBlend(child1, child2, alpha)
                        else:
                            child1,child2 = cxBlend(child1, child2, alpha)
                offspring.append(child1)
                offspring.append(child2)

            # apply mutation to offspring
            for i in range(len(offspring)):
                if np.random.uniform() < mutpb:
                    if mut_type=="normal":
                        if indpb=="random":
                            indpb=np.random.uniform()
                            offspring[i] = mutGaussian(offspring[i],sigma,indpb)
                        else:
                            offspring[i] = mutGaussian(offspring[i],sigma,indpb)
                    elif mut_type=="adaptive":
                        if indpb=="random":
                            indpb=np.random.uniform()
                            offspring[i] = mutGaussianAdaptive(offspring[i],sigma,indpb,tau)
                        else:
                            offspring[i] = mutGaussianAdaptive(offspring[i],sigma,indpb,tau)       

            # evaluate offspring
            invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
            for ind, fit in zip(invalid_ind, pool.map(toolbox.evaluate,invalid_ind)):
                    ind.fitness.values = fit

            # record best solution
            for ind in offspring:
                if ind.fitness.values[0]>hof['max_value']:
                    hof['best_solution'] = ind
                    hof['max_value'] = ind.fitness.values[0]

            # select the next generation of the population
            pop = tools.selTournament(individuals=offspring,k=pop_size,tournsize=tournsize_survival)

            record = stats.compile(pop) if stats else {}
            logbook.record(gen=g+1, nevals=len(invalid_ind), **record, max_hof=hof['max_value'])
            if verbose==1:
                print(logbook.stream)
        pool.close()
        pool.join()

    return hof,logbook

In [9]:
hidden_neurons = 10
individual_size = parameter_count(hidden_neurons)

headless = True
if headless:
    os.environ["SDL_VIDEODRIVER"] = "dummy"

env = Environment(playermode="ai",
                  player_controller=player_controller(hidden_neurons),
                  speed="fastest",
                  enemymode="static",
                  level=2,
                  visuals=False,
                  logs="off")

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("Mean", np.mean)
stats.register("Max", np.max)
stats.register("Std", np.std)

ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5181:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5181:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5181:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5704:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2666:(snd_pcm_open_noupdate) Unknown PCM default


In [10]:
def objective(trial):
    
    params = {'initial_range':trial.suggest_int("initial_range", 1,3,step=1),
              'pop_size':trial.suggest_int("pop_size", 50,50,step=50),
              'n_offspring':trial.suggest_int("n_offspring", 1,5,step=1),
              'tournsize_parent':trial.suggest_int("tournsize_parent", 1,10,step=1),
              'cxpb':trial.suggest_float("cxpb", 0.1, 1., step=0.1),
              'cx_type':trial.suggest_categorical("cx_type",["whole_aritmetic","blend"]),
              'alpha':trial.suggest_categorical("alpha",np.arange(0.1,1.1,0.1).tolist()+["random"]),
              'mutpb':trial.suggest_float("mutpb", 0.1, 1., step=0.1),
              'mut_type':trial.suggest_categorical("mut_type",["normal","adaptive"]),
              'sigma':trial.suggest_float("sigma", 0.5, 3, step=0.5),
              'indpb':trial.suggest_categorical("indpb",np.arange(0.1,1.1,0.1).tolist()+["random"]),
              'tau':trial.suggest_float("tau", 0.1, 1., step=0.1),
              'tournsize_survival':trial.suggest_int("tournsize_survival", 1,10,step=1),
              'ngen':trial.suggest_int("ngen", 30,30,step=20),
              'verbose':trial.suggest_int("verbose", 1,1,step=1)}
    
    global hof_tuning
    iteration_logs = []
    for i in range(3):
        hof,logs = genetic_algorithm(**params)
        logs_pd = pd.DataFrame(logs)
        iteration_logs.append(logs_pd)
        hof_tuning.append(hof)
    
    return pd.concat(iteration_logs)['Max'].mean()


In [ ]:
hof_tuning = []

study = optuna.create_study(study_name="GA", direction="maximize")

study.optimize(objective, n_trials=500)

[I 2023-10-14 23:00:46,648] A new study created in memory with name: GA


gen	nevals	Mean    	Max   	Std    	max_hof
0  	50    	-677.852	-215.4	135.189	-215.4 
1  	138   	-303.016	-191.4	81.5807	-191.4 
2  	129   	-214.672	-174.6	27.7189	-174.6 
3  	129   	-159.676	-106.2	33.4336	-106.2 
4  	125   	-109.444	-76   	23.3432	-76    
5  	135   	-103.192	-76   	14.6603	-76    
6  	130   	-95.956 	-76   	38.4188	-76    
7  	136   	-79.528 	-63.6 	13.8466	-63.6  
8  	133   	-102.236	-63.6 	83.8146	-63.6  
9  	128   	-68.752 	-63.6 	20.675 	-63.6  
10 	137   	-72.956 	-63.6 	37.0618	-63.6  
11 	127   	-67.776 	-63.6 	29.232 	-63.6  
12 	132   	-66.22  	-63.6 	18.34  	-63.6  
13 	132   	-97.384 	-63.6 	80.2857	-63.6  
14 	131   	-66.092 	-63.6 	17.444 	-63.6  
15 	131   	-63.176 	-53   	2.07717	-53    
16 	143   	-75.356 	-53   	42.2717	-53    
17 	134   	-64.944 	-53   	40.718 	-53    
18 	138   	-55.284 	-42.2 	13.5597	-42.2  
19 	133   	-57.572 	-42.2 	26.5677	-42.2  
20 	134   	-62.388 	-42.2 	44.7297	-42.2  
21 	134   	-69.264 	-42.2 	47.1014	-42.2  
22 	135   	

[I 2023-10-14 23:28:18,078] Trial 0 finished with value: -61.853763440859524 and parameters: {'initial_range': 2, 'pop_size': 50, 'n_offspring': 3, 'tournsize_parent': 3, 'cxpb': 0.6, 'cx_type': 'whole_aritmetic', 'alpha': 0.7000000000000001, 'mutpb': 0.7000000000000001, 'mut_type': 'normal', 'sigma': 2.0, 'indpb': 0.9, 'tau': 0.7000000000000001, 'tournsize_survival': 9, 'ngen': 30, 'verbose': 1}. Best is trial 0 with value: -61.853763440859524.


30 	135   	-63.344 	-53.2 	51.2579    	-53.2  
gen	nevals	Mean    	Max   	Std    	max_hof
0  	50    	-687.268	-273.2	123.959	-273.2 
1  	49    	-569.544	-195.4	160.809	-195.4 
2  	50    	-460.62 	-121.2	152.237	-121.2 
3  	48    	-377.192	-119.2	146.26 	-119.2 
4  	50    	-327.384	-159.4	130.267	-119.2 
5  	45    	-307.296	-165.4	105.582	-119.2 
6  	48    	-254.88 	-111  	105.254	-111   
7  	49    	-262.708	-128.8	82.2602	-111   
8  	49    	-280.028	-104.4	131.002	-104.4 
9  	48    	-232.82 	-119  	101.727	-104.4 
10 	49    	-219.608	-128.4	88.5212	-104.4 
11 	45    	-193.944	-100.2	68.4895	-100.2 
12 	49    	-195.948	-101.8	40.8557	-100.2 
13 	50    	-197.708	-110.2	48.6476	-100.2 
14 	50    	-167.58 	-87.8 	47.5545	-87.8  
15 	49    	-189.244	-118.4	33.0954	-87.8  
16 	49    	-173.904	-89.6 	49.0711	-87.8  
17 	49    	-184.936	-29.8 	53.5299	-29.8  
18 	48    	-150.8  	-89.4 	49.5397	-29.8  
19 	50    	-157.596	-93   	45.3745	-29.8  
20 	48    	-154.428	-91   	42.976 	-29.8  
21 	50 

[I 2023-10-14 23:38:41,783] Trial 1 finished with value: -111.22365591397794 and parameters: {'initial_range': 1, 'pop_size': 50, 'n_offspring': 1, 'tournsize_parent': 1, 'cxpb': 0.8, 'cx_type': 'whole_aritmetic', 'alpha': 0.9, 'mutpb': 0.9, 'mut_type': 'adaptive', 'sigma': 2.0, 'indpb': 0.1, 'tau': 1.0, 'tournsize_survival': 3, 'ngen': 30, 'verbose': 1}. Best is trial 0 with value: -61.853763440859524.


30 	49    	-155.5  	-90.2 	63.613 	3.4    
gen	nevals	Mean    	Max   	Std    	max_hof
0  	50    	-694.884	-244.8	137.728	-244.8 
1  	100   	-562.196	-119.8	170.195	-119.8 
2  	100   	-382.956	-119.8	187.345	-119.8 
3  	100   	-432.744	-108  	253.442	-108   
4  	100   	-328.296	-113.6	188.709	-107.8 
5  	100   	-270.06 	-105.8	123.917	-105.8 
6  	100   	-270.668	-100.2	149.381	-100.2 
7  	100   	-309.172	-104.8	172.712	-100.2 
8  	100   	-345.188	-111.2	147.755	-100.2 
9  	100   	-341.068	-123.2	208.108	-63.6  
10 	100   	-421.18 	-106.2	193.191	-63.6  
11 	100   	-320.2  	-60.4 	208.367	-60.4  
12 	100   	-370.552	-44.8 	197.236	-44.8  
13 	100   	-287.676	-44.8 	179.973	-9.2   
14 	100   	-282.64 	-44.8 	191.289	-9.2   
15 	100   	-191.948	-25.2 	161.333	-9.2   
16 	100   	-256.064	-44.8 	208.69 	-9.2   
17 	100   	-203.54 	-44.8 	156.111	-9.2   
18 	100   	-160.616	-44.8 	166.909	-9.2   
19 	100   	-193.228	-44.8 	155.179	-9.2   
20 	100   	-197.82 	-44.8 	169.179	-9.2   
21 	100   	

[I 2023-10-14 23:57:50,376] Trial 2 finished with value: -58.97204301075194 and parameters: {'initial_range': 2, 'pop_size': 50, 'n_offspring': 2, 'tournsize_parent': 9, 'cxpb': 1.0, 'cx_type': 'blend', 'alpha': 0.8, 'mutpb': 0.5, 'mut_type': 'adaptive', 'sigma': 0.5, 'indpb': 0.4, 'tau': 0.4, 'tournsize_survival': 1, 'ngen': 30, 'verbose': 1}. Best is trial 2 with value: -58.97204301075194.


30 	100   	-199.224	-55.2	148.387	-38    
gen	nevals	Mean    	Max   	Std    	max_hof
0  	50    	-706.716	-495.4	70.3064	-495.4 


In [ ]:
"""
params = {'initial_range':,
          'pop_size':,
          'n_offspring':,
          'tournsize_parent':,
          'cxpb':,
          'cx_type':,
          'alpha':,
          'mutpb':,
          'mut_type':,
          'sigma':,
          'indpb':,
          'tau':,
          'tournsize_survival':,
          'ngen':,
          'verbose':}
""""